<img align="right" src="../../additional_data/banner_siegel.png" style="width:1100px;">

# Advanced Xarray

* [**Sign up to the JupyterHub**](https://www.phenocube.org/) to run this notebook interactively from your browser
* **Compatibility:** Notebook currently compatible with the Open Data Cube environments of the University of Wuerzburg
* **Products used**: `s2_l2a_bavaria`
* **Prerequisites**:  Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_jupyter_introduction.ipynb)
    * The basic structure of the eo2cube [satellite datasets](02_eo2cube.ipynb)
    * How to browse through the available [products and measurements](03_products_and_measurements.ipynb) of the eo2cube datacube 
    * How to [load data from the eo2cube datacube](04_loading_data_and_basic_xarray.ipynb) 

## Background

The Python library `xarray` simplifies working with labelled multi-dimension arrays. The library introduces labels in the forms of dimensions, coordinates and attributes on top of `numpy` arrays. This structure allows easier and more effective handling of remote sensing raster data in a Python environment. Therefore, it is essential to fully understand the structure of an `xarray`. A first introduction into the usage of `xarray` within the eo2cube environment was given in ["04_loading_data_and_basic_xarray"](04_loading_data_and_basic_xarray.ipynb). This notebook builds on this gained knowledge and attempts to give a deeper understanding of the `xarray` data structure of raster data. If you are interested in learning more about the structures of the original `xarray`, have a look at this [**"introduction to xarray" notebook**](intro_to_xarray.ipynb) within the "intro_to_python" directory.
To get more information about the `xarray` package, visit the [offical documentation website](http://xarray.pydata.org/en/stable/).

## Description

This notebook introduces users to the `xarray` library within the datacube environment. It aims to deepen the understanding of the `xarray` structure as a container for remote sensing raster data. Also it introduces useful `xarray` functions to effectivly work with raster data in the eo2cube environment. Within this notebook the following topics are covered:

* Application of built-in `xarray` functions for analyzing raster data

***

## Setting up
### Load packages

The `datacube` package is required to query the eo2cube datacube database and load the requested data. The `with_ui_cbk` function from `odc.ui` enables a progress bar when loading large amounts of data. The `xarray` and `numpy` package are needed for the different methods and analysis steps within this notebook. 

In [2]:
import datacube
from odc.ui import with_ui_cbk
import xarray as xr
import numpy as np

### Datacube connection and load data

First we connect to the datacube and load a dumy dataset from the eo2cube. For this we will use the `s2_l2a_bavaria` product. `data_1` contains scenes of the `s2_l2a_bavaria` product from December 2019 to Februray 2020.

In [15]:
data_1 = dc.load(product= "s2_l2a_bavaria",
                 measurements= ["blue", "green", "red"],
                 x= (9.8506165, 11.273325),
                 y= (49.7352601, 50.191334),
                 time= ("2019-12-01", "2020-02-28"),
                 group_by = "solar_day",
                 progress_cbk=with_ui_cbk())

data_1

<xarray.Dataset>
Dimensions:      (time: 54, x: 10310, y: 5284)
Coordinates:
  * time         (time) datetime64[ns] 2019-12-01T10:36:48 ... 2020-02-28T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 11912 11976 12040 12112 12200 ... 0 0 0 0 0
    green        (time, y, x) int16 11632 11736 11696 11752 11776 ... 0 0 0 0 0
    red          (time, y, x) int16 10824 10944 11112 11208 11240 ... 0 0 0 0 0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

<a id='index_array3'></a>
## **Advanced Indexing**
### 1) Temporal Subset

In the earlier tutorial we introduced `isel()`and `sel()` for indexing data. For both methods a **slicing** operator exists. If the function `slice()` is passed onto the index function, the dataset can be sliced. 
The first example uses the slicing by position method to select the first five scenes in `data_1`. The start value is included and the stop value is excluded.

#### I. Using index number

In [19]:
data_1.isel(time=slice(0,5))

<xarray.Dataset>
Dimensions:      (time: 5, x: 10310, y: 5284)
Coordinates:
  * time         (time) datetime64[ns] 2019-12-01T10:36:48 ... 2019-12-08T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 11912 11976 12040 12112 ... 7680 7740 7584
    green        (time, y, x) int16 11632 11736 11696 11752 ... 7428 7440 7472
    red          (time, y, x) int16 10824 10944 11112 11208 ... 7124 7064 7080
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

This example uses the slicing by label method to select the scenes between "2019-12-08" and "2019-12-25". Note, that when using the `slice()` function with the `sel()` method, both start and stop value are included.

#### II. Using `datetime64` data

In [20]:
data_1.sel(time=slice("2019-12-08","2019-12-25"))

<xarray.Dataset>
Dimensions:      (time: 11, x: 10310, y: 5284)
Coordinates:
  * time         (time) datetime64[ns] 2019-12-08T10:26:52 ... 2019-12-25T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 7700 7684 7660 7568 7608 7552 ... 0 0 0 0 0
    green        (time, y, x) int16 7828 7828 7772 7868 7764 7792 ... 0 0 0 0 0
    red          (time, y, x) int16 8116 8068 8068 8036 7944 7912 ... 0 0 0 0 0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

#### III. Using other time dimensions

`xarray` also includes some useful features for the inspection of the time dimension. It allows to easily extract additional information from a dataset. The following code automatically groups the time dimension in seasons ("DJF", "MAM", JJA", "SON"). Since `data_1` only contains scens from winter months, only the label "DJF" will appear. There are a lot of other `time` dimensions arguments, e.g. `month`, `week`, `weekday`, `dayofyear`.

In [21]:
data_1.time.dt.season

<xarray.DataArray 'season' (time: 54)>
array(['DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF',
       'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF',
       'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF',
       'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF',
       'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF',
       'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF'],
      dtype='<U3')
Coordinates:
  * time         (time) datetime64[ns] 2019-12-01T10:36:48 ... 2020-02-28T10:...
    spatial_ref  int32 25832

It is also possible to extract the "day of year" for a time step.

In [22]:
data_1.time.dt.dayofyear

<xarray.DataArray 'dayofyear' (time: 54)>
array([335, 337, 339, 340, 342, 344, 345, 347, 349, 350, 352, 354, 355,
       357, 359, 360, 362, 364, 365,   2,   4,   5,   7,   9,  10,  12,
        14,  15,  17,  19,  20,  22,  24,  25,  27,  29,  30,  32,  34,
        35,  37,  39,  40,  42,  44,  45,  47,  49,  50,  52,  54,  55,
        57,  59])
Coordinates:
  * time         (time) datetime64[ns] 2019-12-01T10:36:48 ... 2020-02-28T10:...
    spatial_ref  int32 25832

### 2) Spatial Subset
It is possible to index and **slice within the x and y dimensions**. The following example selects the value for each band of the pixel in the second colum of the raster and the fifth row of the raster (`x=2,y=5`)

In [23]:
data_1.isel(x=2, y= 5)

<xarray.Dataset>
Dimensions:      (time: 54)
Coordinates:
  * time         (time) datetime64[ns] 2019-12-01T10:36:48 ... 2020-02-28T10:...
    y            float64 5.562e+06
    x            float64 5.607e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time) int16 11912 5316 0 14248 7696 0 ... 417 0 5988 10584 0
    green        (time) int16 12040 4956 0 13496 7992 0 ... 386 0 5952 9744 0
    red          (time) int16 11224 4704 0 13104 7824 0 ... 439 0 6188 9640 0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

### 3) Combining Temporal and Spatial Subset

Again, this method can be combined with the `slice()` operator to do a spatial subset of the dataset based on the position of the pixels. If you know the actual coordinate (x,y) value (extent) of the spatial subset area, use the `sel()` function.
Additionally, this subset can also be sliced in the time dimensions.

The following example subsets the `data_1` by the spatial location of the pixels. Only the pixels from the first to the fifth column and the pixels from the first to the fifth row are included in the output. Also the scenes where filtered in the time dimension between the first and fifth time step.

In [24]:
data_1.isel(time=slice(0,5), x= slice(0,5), y=slice(0,5))

<xarray.Dataset>
Dimensions:      (time: 5, x: 5, y: 5)
Coordinates:
  * time         (time) datetime64[ns] 2019-12-01T10:36:48 ... 2019-12-08T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 5.562e+06 5.562e+06
  * x            (x) float64 5.607e+05 5.607e+05 5.607e+05 5.607e+05 5.608e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 11912 11976 12040 12112 ... 7628 7604 7568
    green        (time, y, x) int16 11632 11736 11696 11752 ... 7924 7872 7764
    red          (time, y, x) int16 10824 10944 11112 11208 ... 7848 7804 7748
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

## **Data Manipulation & Statistics**

This notebook presents some basic built-in functions of the `xarray` library to manipulate and transform data in a `xarray.Dataset`. In this notebook only a fraction of the available `xarray` functions are presented. For a complete overview of all the available functions and tools of the `xarray` package please visit the [documentation website](http://xarray.pydata.org/en/stable/). The [notebook 07](07_basic_analysis.ipynb) will cover this topic with a focus on an application oriented remote sensing approach.

###  1) Statistical Operation

The simple built-in functions allow the user to do simple calculations with a `xarray.Dataset`.
The **basic math** built-in `xarray` functions are:
* `min()`, `max()`
* `mean()`, `median()`
* `sum()`
* `std()`

The following code demonstrates the easy use of the `max()` function to extract the maximum value of the red band in the `data` dataset.

In [25]:
data.red.max()

<xarray.DataArray 'red' ()>
array(16784, dtype=int16)
Coordinates:
    spatial_ref  int32 25832

To apply a function to every value of a specified dimension (e.g. to calculate the mean of every time step) the `dim` argument in the basic math function must be define with the dimension label.

This examples calculates the mean of the `red` band for each pixel (defined by the unique `x`, `y` combination) over every time step. The result is a data array which can be used for further time series visualization and analysis.

In [26]:
data.red.mean(dim=["x", "y"])

<xarray.DataArray 'red' (time: 4)>
array([836.47704301,   0.        , 775.21419741, 854.76094347])
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:54 ... 2020-04-06T10:...
    spatial_ref  int32 25832

This examples works the other way around. It calculates the standard deviation of every pixel (`x`, `y`) over all timesteps of the dataset `data`.

In [27]:
data.red.std(dim="time")

<xarray.DataArray 'red' (y: 5284, x: 10310)>
array([[1137.96023656, 1138.84159895, 1130.61186421, ...,  214.28529464,
         239.32039194,  350.56053329],
       [1201.71572658, 1177.20821756, 1140.74131599, ...,  252.77658119,
         383.54815539,  578.29231363],
       [1276.9972592 , 1251.14937158, 1167.62523418, ...,  400.65477658,
         625.8232578 ,  707.50883387],
       ...,
       [ 396.17735927,  415.43501297,  434.20638814, ...,  278.58605044,
         328.94224417,  303.82838495],
       [ 437.61077169,  469.87737762,  506.74025644, ...,  197.72961336,
         238.70627872,  242.97775104],
       [ 454.52200167,  540.11295115,  611.34360224, ...,  216.61140321,
         194.3429636 ,  208.51918377]])
Coordinates:
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832

Remember, to access the raw `numpy` array that stores the values of the resulting `xarray.DataArrays`, the suffix `.values` is needed. This allows you to work with the "actual" data values.

In [28]:
data.blue.sum(dim=["x","y"]).values

array([26225165045,           0, 34554443445, 29890138833])

### 2) Conditional Operation

Using conditional operation can be very helpful when we need to analyse satellite scenes or pixels lies within our interests. The `where()` function provides the option to **mask** a `xarray.Dataset` based on a logical condition. By default, the function converts all values that match the condition to NaN values. This is extremly useful when applied in combination with a binary mask to mask your data to the desired values. The argument `other` let´s you define a subset value for all values that match the condition (default is `nan`). The argument `drop` drops all values which do not match with the condition.
The following example masks the datatset `data` to only the values which have a reflectance value of greater than 700 in the `red` band.

In [29]:
data.where(data.red > 700)

<xarray.Dataset>
Dimensions:      (time: 4, x: 10310, y: 5284)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:54 ... 2020-04-06T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) float64 493.0 nan nan nan nan ... nan nan nan nan
    green        (time, y, x) float64 628.0 nan nan nan nan ... nan nan nan nan
    red          (time, y, x) float64 805.0 nan nan nan nan ... nan nan nan nan
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

This code subsets all zeros in the dataset `data` with the new value -9999.

In [30]:
data.where(data.red != 0, other = -9999)

<xarray.Dataset>
Dimensions:      (time: 4, x: 10310, y: 5284)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:54 ... 2020-04-06T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 493 302 271 355 265 ... 322 323 238 202 258
    green        (time, y, x) int16 628 394 401 498 405 ... 479 429 322 311 367
    red          (time, y, x) int16 805 459 438 600 436 ... 581 512 409 380 421
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

The implemented `xarray` function `isin()` allows to **test each value** of `xarray.Dataset` or `xarray.DataArray` whether it is in the elements defined within the function. It returns a boolean array which can be used as a mask.
This example checks all the values of the `red` measurement if the value is in an array from 0 to 550.

In [31]:
mask_red = data.red.isin(range(550))
mask_red

<xarray.DataArray 'red' (time: 4, y: 5284, x: 10310)>
array([[[False,  True,  True, ...,  True,  True, False],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ...,  True,  True, False],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],

       [[False,  True,  True, ...,  True,  True, False],
        [False,  True,  True, ...,  True, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ...,  True, False,  True],
        [False, False, False, ...,  True,  True,  True]]])
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:54 ... 2020-04-06T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832

The created mask can easily be combined with the `where()` function to filter the dataset based on the predefined mask. In this case the `data` dataset is masked with previously defined mask `mask_red`, which was based on a logical test if values of the `red` band are within a certain range of values.

In [32]:
data.where(mask_red)

<xarray.Dataset>
Dimensions:      (time: 4, x: 10310, y: 5284)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:54 ... 2020-04-06T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) float64 nan 302.0 271.0 nan ... 238.0 202.0 258.0
    green        (time, y, x) float64 nan 394.0 401.0 nan ... 322.0 311.0 367.0
    red          (time, y, x) float64 nan 459.0 438.0 nan ... 409.0 380.0 421.0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

### 3) Resampling
Working with time series data, resampling is necessary if we want the data product aligns with the temporal window are interested in.

 - **resample()**

The **`resample()` method** allows to summarise the `xarray.Dataset` to bigger or smaller chunks based on a dimension. It handels both upsampling and downsampling. The argument `time` needs to be defined like a datetime-like coordinate. In the following example we resample the `data_1` dataset to a monthly time intervall (`time = "m"`) and than calculate the median value for every resample chunk. _(this process takes a little while to run)_

In [36]:
data_1.resample(time='m').median()

<xarray.Dataset>
Dimensions:      (time: 3, x: 10310, y: 5284)
Coordinates:
  * time         (time) datetime64[ns] 2019-12-31 2020-01-31 2020-02-29
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) float64 2.086e+03 2.074e+03 2.09e+03 ... 0.0 0.0
    green        (time, y, x) float64 2.026e+03 2.068e+03 2.01e+03 ... 0.0 0.0
    red          (time, y, x) float64 1.976e+03 1.952e+03 1.914e+03 ... 0.0 0.0

 - **groupby() method**

The **`groupby()` method** can also be used within the `xarray` library to *aggregate data over time*. Time aggregation arguments can be e.g. "time.year", "time.season", "time.month", "time.week", "time.day".
The code below groups the `data_1` dataset into two groups by year. Therefore, a new "dimension" `year` is created. Then the median for each `year` is calculated. _(this process takes a little while to run)_

In [38]:
data_1.groupby("time.year").median(dim="time")

<xarray.Dataset>
Dimensions:      (x: 10310, y: 5284, year: 2)
Coordinates:
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
  * year         (year) int64 2019 2020
Data variables:
    blue         (year, y, x) float64 2.086e+03 2.074e+03 2.09e+03 ... 0.0 0.0
    green        (year, y, x) float64 2.026e+03 2.068e+03 2.01e+03 ... 0.0 0.0
    red          (year, y, x) float64 1.976e+03 1.952e+03 1.914e+03 ... 0.0 0.0

### 4) Interpolation
Interpolation is a common solution dealing with missing remote sensing data, either caused by coarse temporal resolution of the satellite, high cloud cover or bad quality of the scenes. For example, sometimes a scene of a specfic date is not available in the dataset. With the implemented `interp()` it is possible to **interpolate data** for predefined time steps. The function takes the next usable scene before and after the specified date and interpolates their values (default interpolation method is "linear") to build a new `xarray.Dataset`.

In this example, the `data` dataset has missing scenes on the "2020-04-02" and "2020-04-05". The `interp()` function builds two "new" scenes based on an linear interpolation from the two measurments before and after the new time step.

In [40]:
data.time

<xarray.DataArray 'time' (time: 4)>
array(['2020-04-01T10:26:54.000000000', '2020-04-03T10:16:58.000000000',
       '2020-04-04T10:36:49.000000000', '2020-04-06T10:26:55.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:54 ... 2020-04-06T10:...
    spatial_ref  int32 25832
Attributes:
    units:    seconds since 1970-01-01 00:00:00

In [42]:
data_interp = data.interp(time=["2020-04-02", "2020-04-05"])
data_interp

<xarray.Dataset>
Dimensions:      (time: 2, x: 10310, y: 5284)
Coordinates:
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
  * time         (time) datetime64[ns] 2020-04-02 2020-04-05
Data variables:
    blue         (time, y, x) float64 353.3 216.4 194.2 ... 66.6 56.53 72.2
    green        (time, y, x) float64 450.1 282.4 287.4 ... 90.11 87.03 102.7
    red          (time, y, x) float64 576.9 329.0 313.9 ... 114.5 106.3 117.8
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

The `merge()` function allows to **merge/join** `xarray.Datasets` or variables. By default the `merge()` function uses an "inner" join as merging operation. 
In our example the interpolated `xarray.Dataset` created above is merged to the `data` dataset by using the default `merge()` function.

In [43]:
data.merge(data_interp)

<xarray.Dataset>
Dimensions:      (time: 6, x: 10310, y: 5284)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:26:54 ... 2020-04-06T10:...
  * y            (y) float64 5.562e+06 5.562e+06 5.562e+06 ... 5.51e+06 5.51e+06
  * x            (x) float64 5.607e+05 5.607e+05 ... 6.638e+05 6.638e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) float64 493.0 302.0 271.0 ... 238.0 202.0 258.0
    green        (time, y, x) float64 628.0 394.0 401.0 ... 322.0 311.0 367.0
    red          (time, y, x) float64 805.0 459.0 438.0 ... 409.0 380.0 421.0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

The `xarray` package contains a variety of other useful functions in addition to those shown here. There is a function for almost every operation needed in data analysis. For more information about the `xarray` package visit the [documentation website](http://xarray.pydata.org/en/stable/) or work through the [notebook]() in the "intro_to_python" folder.

The `xarray.Datasets` in the eo2cube datacube environment are a useful and effective structure for handeling remote sensing raster data. In this notebook you learned the basic structure and application methods of `xarray.Datasets`and `xarray.DataArrays`. However, it is very useful to not only work with the "raw" datasets and values. Sometimes it is necessary to get a visual overview of the data. The next [notebook 06](06_plotting.ipynb) will cover how to plot `xarray` raster data nicely and efficient. This is a very useful application, as it is often more convenient to visualize the raster data.

## Recommended next steps

To continue with the beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](01_jupyter_introduction.ipynb)
2. [eo2cube](02_eo2cube_introduction.ipynb)
3. [Search and Load Data](03_data_lookup_and_loading.ipynb)
4. [Xarray I: Data Structure](04_xarrayI_data_structure.ipynb)
5. **Xarray II: Index and Statistics (this notebook)**
6. [Plot](06_plotting_basics.ipynb)
7. [Basic analysis of remote sensing data](07_spatial_analysis_time_series.ipynb)
8. [Parallel processing with Dask](08_parallel_processing_with_dask.ipynb)

***

## Additional information

<font size="2">This notebook for the usage in the Open Data Cube entities of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/), is adapted from [Geoscience Australia](https://github.com/GeoscienceAustralia/dea-notebooks), published using the Apache License, Version 2.0. Thanks! </font>

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** February 2021